In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b7eacf48ba00a041bda60b6c25e74a04c97596843080c43acd23c1944fe9c6f2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 20.6 MB/s eta 0:00:00


In [ ]:
!unzip .streamlit.zip

Archive:  .streamlit.zip
   creating: .streamlit/
  inflating: .streamlit/config.toml  


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files

# Upload a folder from your local machine
uploaded = files.upload()


Saving images.zip to images.zip


In [ ]:
!unzip images.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/58128.jpg        
  inflating: images/58129.jpg        
  inflating: images/5813.jpg         
  inflating: images/58131.jpg        
  inflating: images/58132.jpg        
  inflating: images/58133.jpg        
  inflating: images/58135.jpg        
  inflating: images/58136.jpg        
  inflating: images/58137.jpg        
  inflating: images/58138.jpg        
  inflating: images/58139.jpg        
  inflating: images/5814.jpg         
  inflating: images/58140.jpg        
  inflating: images/58141.jpg        
  inflating: images/58143.jpg        
  inflating: images/58144.jpg        
  inflating: images/58145.jpg        
  inflating: images/58146.jpg        
  inflating: images/58147.jpg        
  inflating: images/58148.jpg        
  inflating: images/58149.jpg        
  inflating: images/5815.jpg         
  inflating: images/58150.jpg        
  inflating: images/58151.jpg        
  inflating: images/58152.jpg        

In [ ]:
%%writefile cv_app.py
from sentence_transformers import SentenceTransformer
import streamlit as st
from PIL import Image
import os
import pinecone

# Initialize Pinecone and create an index if it doesn't exist
pinecone.init(api_key='1407c226-dc8c-4565-9afa-da62f260b3ae', environment='gcp-starter')
index_name = "clip-sim"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=512, metric="cosine", pod_type="s1")

# Image folder
img_folder = 'images'

# Initialize SentenceTransformer model
model = SentenceTransformer('/content/drive/MyDrive/clip-model')

def pinecone_sim_search(query, k=5):
    # Recreate the index object for each search
    index = pinecone.Index(index_name)

    # Encode the query (img/text) using the model
    query_emb = model.encode([query], convert_to_tensor=False, show_progress_bar=False)
    query_emb_list = query_emb.tolist()

    # Perform a similarity search using Pinecone
    results = index.query(query_emb_list, top_k=k)

    # Check if results are None or empty
    if results is None or 'matches' not in results:
        st.write("No results found.")
        return


    # Create a row for images using columns
    images_row = st.columns(len(results['matches']))

    # Display 5 similar images in a single row
    for i, result in enumerate(results['matches']):
        image_id = result['id']
        image_path = os.path.join(img_folder, image_id)
        images_row[i].image(Image.open(image_path).resize((200, 200)), caption=f"ID: {image_id}\nScore: {result['score']}")

# Streamlit app
# Set the page layout to wide
st.set_page_config(layout="wide")

# Center-aligned title
st.title("Similarity Search App")

# Default text input query
default_text_query = "frock for girls"

# Default image query
default_image_query_path = "images/1525.jpg"
default_image_query = Image.open(default_image_query_path)

# Create separate columns for text input and image upload
text_input_col, upload_col = st.columns(2)

# Text input column
with text_input_col:
    pinecone_sim_search(default_text_query)
    text_input = st.text_input("Enter Text Prompt:")
    search_button_text = st.button("Search Text Query")

    if search_button_text:
        # Call the pinecone_sim_search function with the text prompt
        pinecone_sim_search(text_input)

# Image upload column
with upload_col:
    pinecone_sim_search(default_image_query)
    # Allow the user to upload a different image
    uploaded_image = st.file_uploader("Choose an image...", type="jpg")

    if uploaded_image is not None:
        # Call the pinecone_sim_search function with the uploaded image
        pinecone_sim_search(Image.open(uploaded_image))


Overwriting cv_app.py


In [ ]:
!streamlit run cv_app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule debug@4.3.2 che


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.100.198:8501

npx: installed 22 in 3.825s
your url is: https://famous-cycles-judge.loca.lt
2023-12-10 17:11:30.661708: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 17:11:30.661790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 17:11:30.661841: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 17:11:32.974208: W tensorflow/compiler/tf2tensorrt/ut